In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os.path as op
import numpy as np
import seaborn as sns
import pyro

import sys
sys.path.insert(0, '../')

from spectrum.preprocessing import encoders
from spectrum.judge import lca, utils

from spectrum import evaluator

Create some data

In [4]:
claims = dict()
claims['source_id'] = [0, 0, 1, 1]
claims['object_id'] = [0, 1, 1, 0]
claims['value'] = [0, 1, 0, 1]
# claims['domain_size'] = [2, 2, 2, 2]
claims = pd.DataFrame(data=claims)

mask = lca.build_mask(claims)
observation = lca.build_observation(claims)

### Model with observation

In [5]:
utils.print_trace(pyro.poutine.trace(lca.lca_model).get_trace(claims))

s_0 - sampled value 1.0
s_1 - sampled value 0.0
y_0 - sampled value 1
y_1 - sampled value 0
b_0_0 - sampled value 1
b_0_1 - sampled value 0
b_1_1 - sampled value 1
b_1_0 - sampled value 0


In [6]:
trace = pyro.poutine.trace(lca.lca_model).get_trace(claims)

In [7]:
trace.log_prob_sum()

tensor(-2.2423, grad_fn=<AddBackward0>)

In [8]:
data = lca.make_observation_mapper(claims)
conditioned_lca = pyro.condition(lca.lca_model, data=data)

for i in range(3):
    utils.print_trace(pyro.poutine.trace(conditioned_lca).get_trace(claims))
    print('_' * 10)

s_0 - sampled value 1.0
s_1 - sampled value 0.0
y_0 - sampled value 0
y_1 - sampled value 0
b_0_0 - sampled value 0
b_0_1 - sampled value 1
b_1_1 - sampled value 0
b_1_0 - sampled value 1
__________
s_0 - sampled value 1.0
s_1 - sampled value 0.0
y_0 - sampled value 0
y_1 - sampled value 0
b_0_0 - sampled value 0
b_0_1 - sampled value 1
b_1_1 - sampled value 0
b_1_0 - sampled value 1
__________
s_0 - sampled value 0.0
s_1 - sampled value 0.0
y_0 - sampled value 0
y_1 - sampled value 1
b_0_0 - sampled value 0
b_0_1 - sampled value 1
b_1_1 - sampled value 0
b_1_0 - sampled value 1
__________


### Guide

In [9]:
lca.lca_guide?

Signature: lca.lca_guide(claims)
Docstring:
Build a guide for lca_model.

A guide is an approximation of the real posterior distribution p(z|D), 
where z represents hidden variables and D is a training dataset.
A guide is needed to perform variational inference.

Parameters
----------
File:      ~/projects/spectrum/spectrum/judge/lca.py
Type:      function


In [10]:
for i in range(3):
    utils.print_trace(pyro.poutine.trace(lca.lca_guide).get_trace(claims))
    print('_' * 10)

s_0 - sampled value 0.0
s_1 - sampled value 0.0
y_0 - sampled value 1
y_1 - sampled value 0
__________
s_0 - sampled value 0.0
s_1 - sampled value 0.0
y_0 - sampled value 1
y_1 - sampled value 1
__________
s_0 - sampled value 0.0
s_1 - sampled value 0.0
y_0 - sampled value 0
y_1 - sampled value 1
__________


In [11]:
params = pyro.get_param_store()

In [12]:
params['theta_s_0']

tensor(-0.0657, requires_grad=True)

In [13]:
def generate_one_simpleLCA_sample(observation, mask):
    tracer = pyro.poutine.trace(conditioned_lca)
    trace = tracer.get_trace(observation, mask)

    for name, node in trace.nodes.items():
        if node['type'] == 'sample':
            print(f'{node["name"]} - sampled value {node["value"]} ')
    return trace